<a href="https://colab.research.google.com/github/Nagisa1002/scraping/blob/main/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# リポジトリ名
Yahoo Shopping Scraping

## Dependency
requirements

## Usage
1.   スクレイピングする際は#で囲まれた部分をチェック
  *   キーワード(KEYWD)の変更
  *抽出するページ数(PAGE)の変更
  *   抽出する要素(ELE)の変更
  *   ELEのタグ(ELE_TAG)の変更 -> 商品ページのHTMLをチェック
  *   商品情報ページのタグ(DESC_LIST)の変更 ->リンク先のHTMLをチェック
  *   get_elements関数の変更 -> 商品ページのHTMLをチェック
2.   ランタイム→全てのセルを実行
3.   ドライブのマウント(URLに飛んでパスワードを入力)


## Attention
* 商品情報を手に入れるにはその商品のリンクに飛ぶ必要があるが商品によって遷移先が異なるため，適宜DESC_LISTを更新すること．



##References
* CSSセレクタ : https://gammasoft.jp/support/css-selector-for-python-web-scraping/



In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#requirements

!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
import urllib.robotparser
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime 
import re
import sys
from itertools import zip_longest
import time

In [63]:
URL = 'https://shopping.yahoo.co.jp/'
NAME='p' #yahoo's textbox name

##############################################
KEYWD = 'カーディガン' 
PAGE=2
ELE = ['商品名', '値段', '画像', '商品説明', '商品URL']
ELE_TAG = ['._2EW-04-9Eayr > span', '._3-CgJZLU91dR', '._2j-qvZxp4nZn', '._2Qs-G7hnS2-2'] #商品名，値段，画像，商品説明が含まれるHTMLタグ
DESC_DICT = {'paypaymall':'.ItemDescription_text', 'bookoffonline':'body > div >table'}
##############################################


BASE_DIR = '/content/drive/My Drive/scraping'
LOG_DIR = f'{BASE_DIR}/src'
EXEC_TIME = datetime.now().strftime("%Y%m%d-%H%M%S")
SAVE_DIR=f'{LOG_DIR}/keyword_{EXEC_TIME}.csv'

In [56]:
def robot_check(url):
    '''
    confirm whether the scraping is permitted
    '''

    rp = urllib.robotparser.RobotFileParser()
    rp.set_url(URL + 'robots.txt')
    rp.read()

    req_rate = rp.request_rate('*')
    if req_rate is None:
        print('クローリングへの指示書はありません')

        req_URL = rp.can_fetch('*', URL)
        if req_URL == True:
            print('URLの取得も許可されています。これよりスクレイピングを開始します。')
        else:
            print('URLの取得が許可されていません。スクレイピングを中止します')
    else:
        print('クローリングへの指示書があります。問題がないか、利用規約を確認ください。')


In [57]:
def define_option():
  '''
  Chrome Startup Parameters
  Chrome options : http://chrome.half-moon.org/43.html#l3e9a23d
  '''
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  return options

In [58]:
def get_elements(soup, ele_dict):
    """
    pick up elements from html code
    check the HTML code of target page and pull out attritude name you wants to get tags (class, id,...)

    if you want's change or add elements, 

    Returns
    ----------
    dict : {'商品名':[],....} 

    """
    tag = [soup.select(a) for a in ELE_TAG]
    for t, p, i, d in zip_longest(*tag):
      ##############################################
      #title
      if t!=None and t.text!='':
        t=t.text
        if 'クーポン有' in t :
          t=re.split('[/／]', t)[1]
        elif len(t)!=0 :
          t=re.split('[/／]', t)[0]
        ele_dict['商品名'].append(t)
      
      #price
      if p!=None:
        ele_dict['値段'].append(p.text)
      
      #img
      if i!=None and i.get('src')!='https://s.yimg.jp/i/space.gif':
        ele_dict['画像'].append(i.get('src'))
      
      #desc
      if d!=None:
        desc_url=d.get('href')
        ele_dict['商品URL'].append(desc_url)
        desc_html = requests.get(desc_url)
        soup2 = BeautifulSoup(desc_html.content, 'lxml')
        desc_text = soup2.select_one('.mdItemDescription')
        if desc_text==None:
          for i in DESC_DICT.keys():
            if i in desc_url:
              desc_text = soup2.select_one(DESC_DICT[i])
              break

        if desc_text!=None:
          desc_text=desc_text.text
        ele_dict['商品説明'].append(desc_text)

     
      ##############################################

    n0 = len(ele_dict['商品名'])
    print('number of emelemts :　', end='')
    for i in ELE:
      n = len(ele_dict[i])
      if n == n0:
        print('{0} length = {1}'.format(i, len(ele_dict[i])), end='')
      else:
        sys.exit('each list length is different. The number of all list lenght must be same. Check html code．')
    print('')

    return ele_dict

In [59]:
def write_csv(ele_dict):
  ele_list=[ele_dict[i] for i in ELE]
  with open(SAVE_DIR, 'w', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(ELE)
        for i in zip(*ele_list):
          writer.writerow(i)
  print('完了',SAVE_DIR)

In [61]:
def scraping():
    ele_dict ={}
    for  i in ELE:
      ele_dict[i]=[]

    option = define_option()

    driver = webdriver.Chrome('chromedriver', options=option) 
    driver.implicitly_wait(10)
    driver.get(URL)

    # input keyword to textbox and go the result's page 
    elem_keyword = driver.find_element_by_name(NAME)
    elem_keyword.send_keys(KEYWD)
    driver.find_element_by_id('ss_srch_btn').click()
    
    # get url
    cur_url = driver.current_url

    for i in range(PAGE):
      #get html
      r_html = requests.get(cur_url)
      soup = BeautifulSoup(r_html.content, 'lxml')

      #go next page
      cur_url=soup.select_one('.yHvnw7WEUQym > a')
      cur_url=cur_url.get('href')

      ele_dict = get_elements(soup, ele_dict)
    write_csv(ele_dict)



In [62]:
def main():
    #robot_check(url) 
    scraping()

if __name__ == '__main__':
    main()

number of emelemts :　商品名 length = 30値段 length = 30画像 length = 30商品説明 length = 30商品URL length = 30
number of emelemts :　商品名 length = 60値段 length = 60画像 length = 60商品説明 length = 60商品URL length = 60
完了 /content/drive/My Drive/scraping/src/keyword_20210607-082217.csv
